<a href="https://colab.research.google.com/github/neilalden/esentry-server/blob/master/water_quality_time_series_forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [128]:
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import sklearn
import pickle
import joblib
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from google.colab import drive
import warnings
warnings.filterwarnings('ignore')

In [129]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [130]:
df_path_1 = "/content/drive/MyDrive/Colab Notebooks/datasets/1.csv"
df_path_2 = "/content/drive/MyDrive/Colab Notebooks/datasets/2.csv"
models_path = "/content/drive/MyDrive/Colab Notebooks/models"
df_1 = pd.read_csv(df_path_1, encoding= 'unicode_escape')
df_2 = pd.read_csv(df_path_2, encoding= 'unicode_escape')
main_df = pd.concat([df_1, df_2])
main_df = main_df.dropna()

In [131]:
main_df

,Date,Temp,pH,Sal,Amm,Nit,Chl
0,4/29/2022 14:05,26.66,11.44,-0.02,45.98,44.98,0.0
1,4/29/2022 14:06,26.86,11.44,-0.02,45.90,44.93,0.0
2,4/29/2022 14:07,26.76,11.44,-0.02,45.77,44.92,0.0
3,4/29/2022 14:08,26.75,11.43,-0.02,45.81,45.98,0.0
4,4/29/2022 14:09,26.36,11.43,-0.02,45.80,45.95,0.0
...,...,...,...,...,...,...,...
47,4/29/2022 15:44,24.69,9.42,-0.02,45.19,45.67,0.0
48,4/29/2022 15:45,24.69,9.42,-0.02,45.17,45.73,0.0
49,4/29/2022 15:46,24.69,9.42,-0.02,45.17,45.73,0.0
50,4/29/2022 15:47,24.67,9.41,-0.02,45.11,45.64,0.0


In [132]:
params = ["Temp", "pH", "Sal","Amm",	"Nit",	"Chl"]
dfs = []

In [133]:
for label in params:
  df = pd.DataFrame()
  df = main_df[[label]]
  df[f'{label} 1'] = df[label].shift(+1)
  df[f'{label} 2']= df[label].shift(+2)
  df[f'{label} 3']= df[label].shift(+3)
  df[f'{label} 4']= df[label].shift(+4)
  df[f'{label} 5']= df[label].shift(+5)
  df = df.dropna()
  df.tail(7)
  dfs.append(df)

In [134]:
dfs[0]

,Temp,Temp 1,Temp 2,Temp 3,Temp 4,Temp 5
5,26.26,26.36,26.75,26.76,26.86,26.66
6,26.86,26.26,26.36,26.75,26.76,26.86
7,26.46,26.86,26.26,26.36,26.75,26.76
8,26.56,26.46,26.86,26.26,26.36,26.75
9,26.56,26.56,26.46,26.86,26.26,26.36
...,...,...,...,...,...,...
47,24.69,24.70,24.71,24.71,24.72,24.73
48,24.69,24.69,24.70,24.71,24.71,24.72
49,24.69,24.69,24.69,24.70,24.71,24.71
50,24.67,24.69,24.69,24.69,24.70,24.71


In [135]:
models = []

In [136]:
i = 0
for label in params:
  y = np.array(dfs[i][label])
  x = np.array(dfs[i].drop([label], 1))
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state=1387)
  linear = LinearRegression()

  pipeline = make_pipeline(MinMaxScaler(),linear)
  model = pipeline.fit(x_train, y_train)


  # scaler = MinMaxScaler().fit(x_train)
  # x_train = scaler.transform(x_train) 
  # x_test = scaler.transform(x_test)
  # linear.fit(x_train, y_train)
  # y_pred = linear.predict(x_test)

  acc = model.score(x_test, y_test)
  models.append(model)
  i += 1
  print(f"{label} model accuracy :", acc)
  # plt.rcParams["figure.figsize"] = (15,5)
  # plt.plot(y_test, '-o',label="actual")
  # plt.plot(y_pred, '-o', label="predict")
  # plt.ylabel(label)
  # plt.legend(loc="upper right")
  # plt.show()

Temp model accuracy : 0.7670783071410862
pH model accuracy : 0.9157158482792489
Sal model accuracy : 1.0
Amm model accuracy : 0.6365586060587167
Nit model accuracy : 0.8151757381265539
Chl model accuracy : 1.0


In [137]:
i = 0
for m in models:
  pickle.dump(m, open(f"{models_path}/{params[i]}_lr_tsf.pkl", 'wb'))
  i += 1